# AAA

## chargers
- openchargemap => te weinig laders
- chargefinder => website scrapen en niet accuraat
- google => geen cegeka laders
- luminus => ok voor publieke laders, prive laders enkel hoeveel en hoeveel kapot, niet vrij/busy
- chargemap 2 => login lukt niet
- alois heeft ook nog 1 gevonden

In [7]:
import requests
from pprint import pprint
from slack_sdk.webhook import WebhookClient
import os

In [3]:
slack_url = os.environ["SLACK_WEBHOOK_URL"]

In [8]:
class Charger:
    def __init__(self, name, adress, lat, long, availability, max_watt, price):
        self.name = name
        self.adress = adress
        self.lat = lat
        self.long = long
        self.availability = availability
        self.max_watt = max_watt
    
    def is_available(self):
        return self.availability == 'available'
        
    def __str__(self):
        return f"{self.name} ({self.adress}): {self.availability} (max {self.max_watt})"

In [3]:
def get_availability(availability):
        match(availability):
            case 1:
                return 'available'
            case 2:
                return 'broken'
            case 3:
                return 'in use'

def mapChargers(charger):
    return Charger(
        charger['evseId'], 
        charger['location']['address'], 
        charger['latitude'], 
        charger['longitude'], 
        get_availability(charger['globalStatus']), 
        charger['maxPower'],
        0
  )
# name = charger['evseId']
# adress = charger['location']['address']
# lat = charger['latitude']
# long = charger['longitude']
# availability = charger['globalStatus']
# max_watt = charger['maxPower']
# price = charger['simpleTariff'] # obj, needs calculation

In [6]:
def refreshData():
    urls = [
        #r'https://luminus.evc-net.com/api/ajax?requests=%7B%220%22%3A%7B%22handler%22%3A%22%5C%5CLMS%5C%5CEV%5C%5CAsyncServices%5C%5CDashboardAsyncService%22%2C%22method%22%3A%22spotsStatusPointData%22%2C%22params%22%3A%7B%22deviceIds%22%3A%5B156888067%2C156888068%5D%2C%22tariffProvider%22%3A250%7D%7D%7D&metricKey=DeviceMap_1037'
        r'https://luminus.evc-net.com/api/ajax?requests=%7B%220%22%3A%7B%22handler%22%3A%22%5C%5CLMS%5C%5CEV%5C%5CAsyncServices%5C%5CDashboardAsyncService%22%2C%22method%22%3A%22spotsStatusPointData%22%2C%22params%22%3A%7B%22deviceIds%22%3A%5B154435032%2C154435033%5D%2C%22tariffProvider%22%3A250%7D%7D%7D&metricKey=DeviceMap_1037'
        #todo voeg andere chargers ook toe
    ]

    # if not chargers:
    #     return
    
    # onlinePastChargers = {}
    # for charger in chargers:
    #     if charger.is-online():
    #         onlinePastChargers[charger.name] = charger
        
    responses = [requests.get(url) for url in urls]
    data = [response_charger 
            for response_chargers in responses 
            for response_charger in response_chargers.json()[0]]
    return data

In [10]:
# data = refreshData()
chargers = [mapChargers(charger) for charger in data]

for charger in chargers:
    if charger.is_available():
        print(charger)

BEALLEGO0020692 (Aststraat 1): available (max 11.0 kW)


In [9]:
webhook = WebhookClient(slack_url)

In [12]:
def map_block(block):
    return {
                "type": "section",
                "text": {
                    "type": "mrkdwn",
                    "text": block
                }
            }

def send_message_to_slack(text, blocks=None):
    if blocks is None:
        webhook.send(text=text)
        return

    blocks = [map_block(block) for block in blocks]
    webhook.send(text=text, blocks=blocks)

In [14]:
send_message_to_slack("# h1 Heading 8-)")

In [77]:
# def compare():
#     for charger in chargers if charger.is_online():
#         if charger.name not in onlinePastChargers:
#             print(f"following charger is now available: {charger}")

In [ ]:
#todo start server die pollt en availability vergelijk
import schedule
import time

schedule.every(1).minutes.do(refreshData)

In [ ]:
# # Slack
# ## Setup
# from slack_sdk.webhook import WebhookClient
# import os
# webhook = WebhookClient(os.environ["SLACK_WEBHOOK_URL"])
# ## function
# def send_message_to_slack(text):
#     webhook.send(text=text)
# ## example
# send_message_to_slack("tung tung tung tung tung tung sahur")